In [1]:
__version__ = '0.3'
__author__  = "Robert Matern (r.matern@stud.uni-hannover.de)"
__date__    = ''
__url__     = ''
__copyright__ = "(C) 2015 Robert Matern"

In [2]:
# import pandas as pd
import numpy as np

In [3]:
# Daten (Parameter) übernehmen
%run Data1.ipynb

In [4]:
import matplotlib.pyplot as plt
import pydecode
%matplotlib inline

#Dynamisches Programm

Die normale Modellformulierung des Auftragsannahmeproblems im Revenue Management von Instandhaltungsprozessen:

$$V(\textbf{c}, t) = \sum_{j \in \mathcal{J}}p_{j}(t)\max(V(\textbf{c}, t-1), r_{j} + V(\textbf{c}-\textbf{a}_j, t-1)) + p_{0}(t)V(\textbf{c}, t-1) $$

$$= V(\textbf{c}, t-1) + \sum_{j \in \mathcal{J}}p_{j}(t)(r_j - V(\textbf{c}, t-1) + V(\textbf{c}-\textbf{a}_j, t-1))$$

## Algorithmus

In [5]:
def Value(conditions):
    values = [] 
    for i in range(len(conditions)):
        if conditions[i][1] == 0:
            values = values + [[conditions[i], 0]]
        else:
            values = values + [[conditions[i], None]]
    return values

values = Value(conditions)
values

[[[(0, 0), 2], None],
 [[(0, 0), 1], None],
 [[(0, 0), 0], 0],
 [[(0, 1), 2], None],
 [[(0, 1), 1], None],
 [[(0, 1), 0], 0],
 [[(1, 0), 2], None],
 [[(1, 0), 1], None],
 [[(1, 0), 0], 0],
 [[(1, 1), 2], None],
 [[(1, 1), 1], None],
 [[(1, 1), 0], 0],
 [[(2, 0), 2], None],
 [[(2, 0), 1], None],
 [[(2, 0), 0], 0],
 [[(2, 1), 2], None],
 [[(2, 1), 1], None],
 [[(2, 1), 0], 0]]

In [6]:
against_probs

array([ 0.4,  0.4,  1. ])

In [7]:
sum = 0
for j in products[1:]:
    sum = sum + probs[j][1]*revenues[j]

print sum

70.0


In [8]:
from itertools import product
conditions = [] 
array = []
for h in resources[1:]:
    array = array + [range(capacities[h]+1)]
for subset in product(*array):
    conditions = conditions + [subset]

table = [[0 for w in range(len(times))] for j in xrange(len(conditions))]

print table

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


In [9]:
def DP(products, resources, capacities, consumtions, revenues, probs, times):
    cache=[]
    if times.any()==0:
        print times
        
    for t in range(len(times)):
        value = DP(products, resources, capacities, consumtions, revenues, probs, times[1:]) + sum(probs[j][t]*(revenues[j]-DP(products, resources, capacities, consumtions, revenues, probs, times[1:])+DP(products, resources, capacities-consumtions[j], consumtions, revenues, probs, times[1:])))
            
DP(products, resources, capacities, consumtions, revenues, probs, times)

[0]
[]


IndexError: index 5 is out of bounds for axis 0 with size 3

## 0/1 Knapsack lt. http://rosettacode.org/wiki/Knapsack_problem/0-1#Python

Definition der Parameter

In [10]:
print("Produkte j:", products)            
print("Ressourcen h:", resources)         
print("Kapazitaet c[h]:", capacities)           
print("Verbrauch a[j]:", consumtions)       
print("Ertrag r[j]:", revenues)         
print("Buchungszeit t:", times)             
print( "Wkeit der Anfrage p[j, t]:", probs)             
print("Wkeit keiner Anfrage p[0, t]", against_probs)    

('Produkte j:', array([0, 1, 2], dtype=uint8))
('Ressourcen h:', array([0, 1, 2], dtype=uint8))
('Kapazitaet c[h]:', array([0, 2, 1], dtype=uint8))
('Verbrauch a[j]:', array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8))
('Ertrag r[j]:', array([  0, 100, 200], dtype=uint8))
('Buchungszeit t:', array([2, 1, 0], dtype=uint16))
('Wkeit der Anfrage p[j, t]:', array([[ 0. ,  0. ,  0. ],
       [ 0.5,  0.5,  0. ],
       [ 0.1,  0.1,  0. ]]))
('Wkeit keiner Anfrage p[0, t]', array([ 0.4,  0.4,  1. ]))


In [11]:
def total_value(items, max_weight):
    return  sum([x[2] for x in items]) if sum([x[1] for x in items]) < max_weight else 0
 
cache = {}
def solve(items, max_weight):
    if not items:
        return ()
    if (items,max_weight) not in cache:
        head = items[0]
        tail = items[1:]
        include = (head,) + solve(tail, max_weight - head[1])
        dont_include = solve(tail, max_weight)
        if total_value(include, max_weight) > total_value(dont_include, max_weight):
            answer = include
        else:
            answer = dont_include
        cache[(items,max_weight)] = answer
    return cache[(items,max_weight)]
 
items = (
    ("map", 9, 150), ("compass", 13, 35), ("water", 153, 200), ("sandwich", 50, 160),
    ("glucose", 15, 60), ("tin", 68, 45), ("banana", 27, 60), ("apple", 39, 40),
    ("cheese", 23, 30), ("beer", 52, 10), ("suntan cream", 11, 70), ("camera", 32, 30),
    ("t-shirt", 24, 15), ("trousers", 48, 10), ("umbrella", 73, 40),
    ("waterproof trousers", 42, 70), ("waterproof overclothes", 43, 75),
    ("note-case", 22, 80), ("sunglasses", 7, 20), ("towel", 18, 12),
    ("socks", 4, 50), ("book", 30, 10),
    )
max_weight = 400
 
solution = solve(items, max_weight)
print "items:"
for x in solution:
    print x[0]
print "value:", total_value(solution, max_weight)
print "weight:", sum([x[1] for x in solution])

TypeError: 'numpy.float64' object is not callable

In [12]:
try:
    xrange
except:
    xrange = range
 
def totalvalue(comb):
    ' Totalise a particular combination of items'
    totwt = totval = 0
    for item, wt, val in comb:
        totwt  += wt
        totval += val
    return (totval, -totwt) if totwt <= 400 else (0, 0)
 
items = (
    ("map", 9, 150), ("compass", 13, 35), ("water", 153, 200), ("sandwich", 50, 160),
    ("glucose", 15, 60), ("tin", 68, 45), ("banana", 27, 60), ("apple", 39, 40),
    ("cheese", 23, 30), ("beer", 52, 10), ("suntan cream", 11, 70), ("camera", 32, 30),
    ("t-shirt", 24, 15), ("trousers", 48, 10), ("umbrella", 73, 40),
    ("waterproof trousers", 42, 70), ("waterproof overclothes", 43, 75),
    ("note-case", 22, 80), ("sunglasses", 7, 20), ("towel", 18, 12),
    ("socks", 4, 50), ("book", 30, 10),
    )
 
def knapsack01_dp(items, limit):
    table = [[0 for w in range(limit + 1)] for j in xrange(len(items) + 1)]
 
    for j in xrange(1, len(items) + 1):
        item, wt, val = items[j-1]
        for w in xrange(1, limit + 1):
            if wt > w:
                table[j][w] = table[j-1][w]
            else:
                table[j][w] = max(table[j-1][w],
                                  table[j-1][w-wt] + val)
 
    result = []
    w = limit
    for j in range(len(items), 0, -1):
        was_added = table[j][w] != table[j-1][w]
 
        if was_added:
            item, wt, val = items[j-1]
            result.append(items[j-1])
            w -= wt
 
    return result
 
 
bagged = knapsack01_dp(items, 400)
print("Bagged the following items\n  " +
      '\n  '.join(sorted(item for item,_,_ in bagged)))
val, wt = totalvalue(bagged)
print("for a total value of %i and a total weight of %i" % (val, -wt))

Bagged the following items
  banana
  compass
  glucose
  map
  note-case
  sandwich
  socks
  sunglasses
  suntan cream
  water
  waterproof overclothes
  waterproof trousers
for a total value of 1030 and a total weight of 396


In [16]:
def test_value(products, capacities, consumtions, revenues, probs, times):
    cache = []
    for i in products[1:]:
        for t in times:
            if times.any()==0:
                continue
            else:
                reject = test_value(products, capacities, consumtions, revenues, probs[t+1:], times[t+1:])
                accept = test_value(products, capacities-consumtions[i], consumtions, revenues, probs[t+1:], times[t+1:])
            value = revenues[i] #+ sum(probs[i][t] * (revenues[i] - reject + accept) for i in products[1:])
            cache.append(value)
            print cache
    return



test_value(products, capacities, consumtions, revenues, probs, times)

[100]
[100, 100]
[100]
[100, 100]
[100, 100, 200]
[100, 100, 200, 200]
[100]
[100, 100]
[100, 100, 200]
[100, 100, 200, 200]
[100, 100, 100]
[100, 100, 100, 200]
[100, 100, 100, 200, 200]
[100]
[100, 100]
[100, 100, 200]
[100, 100, 200, 200]
[100]
[100, 100]
[100, 100, 200]
[100, 100, 200, 200]
[100, 100, 100, 200, 200, 200]


In [17]:
def test_value(products, capacities, consumtions, revenues, probs, times):
    accept = np.zeros(shape=(30,len(times)))
    for i in products[1:]:
        for t in range(len(times)):
            if all(capacities[1:])==True: #Prüft nur, ob alle Zahlen 0 sind..
                if any(times):
                    new_times = times[t+1:]
                else:
                    break
                print(new_times)
                new_capacities = capacities-consumtions[i+1]
                print(new_capacities)
                accept[i+1][t] = probs[i+1][t] * ( revenues[i+1] + test_value(products, new_capacities, 
                                                                     consumtions, revenues, probs, new_times))
            else:
                break
            break
        break
    return accept


test_value(products, capacities, consumtions, revenues, probs, times)

[1 0]
[0 2 0]


ValueError: setting an array element with a sequence.

In [18]:
#@Memo
def Value(Product, Resource, Capacity, Consumption, Revenue, Time, Prob, Against_Prob):
    #accept = np.zeros(len(j)+1)                               # Funktioniert
    #reject = np.zeros(len(j)+1)                               # Funktioniert
    accept = []
    reject = []
    for i in Product:                                   # 0 -> 1 ; 1 -> 2 ; usw.
        for t in range(len(Time)):
            accept.append(Prob[i+1][t] * ( Revenue[i+1] + Value(Product, Resource, Capacity-Consumption[i], 
                                                                Consumption, Revenue, Time[t+1:], Prob, Against_Prob) ))
            reject.append(Prob[i+1][t] * Value(j==i+1, c[i], a[i], r[i], t-1, p[i]))
        if reject[i] > accept[i]: #Was mit größer gleich?
            return reject
        else:
            return accept

Value(products, resources, capacities, consumtions, revenues, times, probs, against_probs)

IndexError: list index out of range

In [19]:
times
chart = pydecode.ChartBuilder(times)
chart.init(times[1:len(times)])
chart.set(times[0],
          [[times[1]], [times[2]], [times[3]], [times[4]]],
          labels=[1,2,3,4])
pydecode.draw(chart.finish(), labels=True)

ValueError: Buffer dtype mismatch, expected 'long' but got 'unsigned short'

In [106]:
def simple_hypergraph():
    """
    Create a simple hypergraph.
    """
    enc = np.arange(7)                                          # array([0, 1, 2, 3, 4, 5])
    c = pydecode.ChartBuilder(enc, np.arange(10))               # <pydecode._pydecode.ChartBuilder at 0x105604150>

    c.init(enc[:4])                                             # None
    
    c.set_t(enc[4], enc[0:2], enc[1:3], labels=np.arange(2))    # None
    
    c.set_t(enc[5], np.repeat(enc[4], 1), enc[[3]], labels=np.array([2])) # None
    
    c.set_t(enc[6], np.repeat(enc[5], 1), enc[[4]], labels=np.array([3])) # None
    
    dp = c.finish()                                             # <pydecode._pydecode.Hypergraph object at 0x1079d2aa0>
    # for edge in hypergraph.edges:
    #     assert edge.label in ["0", "1", "2", "3", "4"]
    return dp

hypergraph = simple_hypergraph()

for vertex in hypergraph.vertices:
    print vertex.id, vertex.is_terminal
    for edge in vertex.edges:
        print "\t", edge.id, edge.label

NameError: global name 'pydecode' is not defined

In [ ]:
#kReject, kAccept = 0, 1
#operations = np.array([kReject, kAccept])
#offsets = np.array([[1,0], [0,1], [1,1]]) 
#op_names = np.array(["Reject",  "Accept"])

In [ ]:
from functools import wraps
def Memo(func):
    '''A Memoizing Decorator. The idea of memoized function is that it caches its return values. If you call it a
    second time with the same parameters, it will simply return the cached value. You can certainly (allerdings) 
    put this sort of caching logic inside your function, but memo function is a more reusable (Mehrweg-) solution.
    '''
    cache = {}                           # Stored subproblem solutions
    @wraps(func)                         # Make wrap look like func
    def wrap(*args):                     # The memoized wrapper
        if args not in cache:            # Not already computed?
            cache[args] = func(*args)    # Compute & cache the solution
        return cache[args]               # Return the cached solution
    return wrap  

In [ ]:
import networkx as nx

In [ ]:
G=nx.Graph()
G.add_node("spam")
G.add_edge(1,2)
print(G.nodes())
[1, 2, 'spam']
print(G.edges())
[(1, 2)]

In [ ]:
def test_hypergraph():
    """
    Create a test hypergraph. (C) PyDecode
    """
    enc = np.arange(6) # [0 1 2 3 4 5]

    c = pydecode.ChartBuilder(enc, np.arange(10)) # <pydecode._pydecode.ChartBuilder object at 0x10a06d550>

    c.init(enc[:4])

    c.set_t(enc[4], enc[0:2], enc[1:3], labels=np.arange(2))
    c.set_t(enc[5], np.repeat(enc[4], 1), enc[[3]], labels=np.array([2]))
    
    dp = c.finish() # <pydecode._pydecode.Hypergraph object at 0x10a164d60>

    # for edge in hypergraph.edges:
    #     assert edge.label in ["0", "1", "2", "3", "4"]
    return dp

In [ ]:
import pydecode
import pydecode.test.utils
graph = test_hypergraph()
weights = np.random.randint(10, size=(len(graph.edges)))
print weights
pydecode.draw(graph, weights)

In [ ]:
inside = pydecode.inside(graph, np.array(weights, dtype=np.double),
                weight_type=pydecode.LogViterbi)
pydecode.draw(graph, weights, inside)

In [ ]:
path = pydecode.best_path(graph, weights * 1.)
pydecode.draw(graph, weights, inside, paths=[path])

In [ ]:
items = np.arange(100)
#print items
chart = pydecode.ChartBuilder(items)   

i=1
n=0
m=2

chart.init(items[0:m]) # Die Quadrate werden erstellt

#for i in range(m):

chart.set_t(items[10],                  # 1 ist verbunden...
            items[n:m],                # ...mit den Ovalen.
            labels=np.arange(m)
           )       # Bezeichnung der Ovale
chart.set_t(items[10],                  # 1 ist verbunden...
            items[n:m],                # ...mit den Ovalen.
            labels=np.arange(m)
           )       # Bezeichnung der Ovale
pydecode.draw(chart.finish(), labels=True)